Hough Transform Example
=======================

In [1]:
import bqplot.pyplot as plt
import bqplot as bq
import numpy as np

## Define two lines

In [201]:
m1 = 3
c1 = 30

m2 = 1.4
c2 = 40

x = np.linspace(0, 100, 100)

y1 = m1*x + c1
y2 = m2*x + c2

In [202]:
plt.figure() 
plt.plot(x, y1)
plt.plot(x, y2)
plt.show()

## Build $\theta$ and $r$ arrays from known $(x_i,y_i)$ points

In [203]:
theta1 = np.tile(np.linspace(0, 2*np.pi, 200).reshape(-1,1), len(x))
r1 = x*np.cos(theta1) + y1*np.sin(theta1)

## Plot in a Histogram

In [204]:
hist_2d, r1_edges, theta1_edges = np.histogram2d(r1.ravel(), 
                                                 theta1.ravel(), 
                                                 bins=100)
hist_2d

0.,0.,0.,⋯,0.,0.,0.
0.,0.,0.,⋯,0.,0.,0.
0.,0.,0.,⋯,0.,0.,0.
⋮,⋮,⋮,⋱,⋮,⋮,⋮
0.,0.,0.,⋯,0.,0.,0.
0.,0.,0.,⋯,0.,0.,0.
0.,0.,0.,⋯,0.,0.,0.


## Draw 2D Histogram of $r,\theta$

In [205]:
plt.figure()
plt.heatmap(hist_2d, x=r1_edges, y=np.degrees(theta1_edges), cmap="viridis", tooltip=bq.Tooltip(fields=['midpoint']))
plt.xlabel("r")
plt.ylabel("theta")
plt.show()

## Extract the top $n$ indices by frequency

In [206]:
max_indices = np.vstack(np.unravel_index(np.argsort(hist_2d, axis=None), hist_2d.shape)).T
max_indices

0,0
62,76
62,75
⋮,⋮
50,45
51,44
48,94


In [207]:
maxima = max_indices[-1:-5:-1]
maxima

48,94
51,44
50,45
49,95


In [208]:
i_r1, j_theta1 = maxima[0]
r1_fit = r_edges[i_r1]
theta1_fit = theta_edges[j_theta1]

In [209]:
plt.figure()
plt.plot(x, (r1_fit - x*np.cos(theta1_fit))/np.sin(theta1_fit), "r--", labels=['Fit'])
plt.plot(x, y1, labels=['True'])
plt.legend()
plt.show()